## library imports

In [1]:
import torch
import transformers
import onnx
import onnxruntime.training.onnxblock as onnxblock
from datasets import load_dataset
import json

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## generating artifacts

In [2]:
# model = transformers.AutoModel.from_pretrained('google/mobilebert-uncased')
from transformers import MobileBertConfig, MobileBertModel 
config = MobileBertConfig(num_hidden_layers=4)
model = MobileBertModel(config)
model_name = 'mobilebert-uncased'

In [3]:
# create the random input

# expects
# input_ids = torch.LongTensor of shape (batch size, seq len)
# attention_mask = torch.FloatTensor of shape (batch size, seq len)
# token_type_ids = torch.LongTensor of shape (bs, seq len)

num_seq = 2
seq_len = 150
vocab = 20000
input_ids = torch.randint(vocab, (num_seq, seq_len), requires_grad=False)
attention_mask = torch.ones((num_seq, seq_len), dtype=torch.float, requires_grad=False)
token_type_ids = torch.ones((num_seq, seq_len), dtype=torch.long, requires_grad=False)


In [4]:
torch.onnx.export(model, (input_ids, attention_mask, token_type_ids),
                  f"training_artifacts/{model_name}.onnx", 
                  input_names=["input_ids", "attention_mask", "token_type_ids"],
                  output_names=["output"],
                   dynamic_axes={
                     "input_ids": {0: "num_seq"},
                     "attention_mask": {0: "num_seq"},
                     "token_type_ids": {0: "num_seq"}
                   },
                   export_params=True, 
                   do_constant_folding=False,
                   training=torch.onnx.TrainingMode.TRAINING)

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:967: UserWarning: Warning: ONNX export of embedding with padding_idx >= 0 for training mode. ONNX does not support not updating the embedding vector at padding_idx during training.
  warnings.warn(
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/_internal/jit_utils.py:306: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:1186: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(


In [3]:
num_seq = 1000
seq_len = 150
vocab = 20000
input_ids = torch.randint(vocab, (num_seq, seq_len), requires_grad=False)
attention_mask = torch.ones((num_seq, seq_len), dtype=torch.float, requires_grad=False)
token_type_ids = torch.ones((num_seq, seq_len), dtype=torch.long, requires_grad=False)

torch.onnx.export(model, (input_ids, attention_mask, token_type_ids),
                  f"training_artifacts/{model_name}.onnx", 
                  input_names=["input_ids", "attention_mask", "token_type_ids"],
                  output_names=["output"])

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/_internal/jit_utils.py:306: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [4]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

for name, buffer in model.named_buffers():
    print(name)

embeddings.word_embeddings.weight True
embeddings.position_embeddings.weight True
embeddings.token_type_embeddings.weight True
embeddings.embedding_transformation.weight True
embeddings.embedding_transformation.bias True
embeddings.LayerNorm.bias True
embeddings.LayerNorm.weight True
encoder.layer.0.attention.self.query.weight True
encoder.layer.0.attention.self.query.bias True
encoder.layer.0.attention.self.key.weight True
encoder.layer.0.attention.self.key.bias True
encoder.layer.0.attention.self.value.weight True
encoder.layer.0.attention.self.value.bias True
encoder.layer.0.attention.output.dense.weight True
encoder.layer.0.attention.output.dense.bias True
encoder.layer.0.attention.output.LayerNorm.bias True
encoder.layer.0.attention.output.LayerNorm.weight True
encoder.layer.0.intermediate.dense.weight True
encoder.layer.0.intermediate.dense.bias True
encoder.layer.0.output.dense.weight True
encoder.layer.0.output.dense.bias True
encoder.layer.0.output.LayerNorm.bias True
encoder.

In [5]:
class MobileBERTWithLoss(onnxblock.TrainingModel):
    def __init__(self):
        super().__init__()
        self.loss = onnxblock.loss.CrossEntropyLoss()

    def build(self, loss_node_input_name):
        print('inside mobilebertwithloss build')
        return self.loss(loss_node_input_name)


# Load the model from the exported inference ONNX file.
onnx_model = onnx.load(f"training_artifacts/{model_name}.onnx")
print('done loading onnx model')
eval_model = None
optimizer_model = None

training_block = MobileBERTWithLoss()
for name, param in model.named_parameters():
    if param.requires_grad:
        training_block.requires_grad(name)
    else:
        training_block.requires_grad(name, False)

training_block.requires_grad('embeddings.position_ids', False)

print('done creating trainingblock')

inference_model_output_name = "output"
with onnxblock.onnx_model(onnx_model) as model_accessor:
    print('within "with" block')
    loss_output_name = training_block(inference_model_output_name)
    print('training block initialized')
    eval_model = model_accessor.eval_model
    print('eval model created')

print('inference model done')

optimizer_block = onnxblock.optim.AdamW()
with onnxblock.onnx_model() as model_accessor:
    optimizer_outputs = optimizer_block(training_block.parameters())
    optimizer_model = model_accessor.model
print('optimizer model done')

done loading onnx model
done creating trainingblock
within "with" block
inside mobilebertwithloss build


2023-03-30 16:28:45.468425642 [I:onnxruntime:Default, graph.cc:3545 CleanUnusedInitializersAndNodeArgs] Removing initializer '/embeddings/Cast_output_0'. It is no longer used by any node.
2023-03-30 16:28:45.468469340 [I:onnxruntime:Default, graph.cc:3545 CleanUnusedInitializersAndNodeArgs] Removing initializer '/embeddings/Sub_output_0'. It is no longer used by any node.
2023-03-30 16:28:45.468494639 [I:onnxruntime:Default, graph.cc:3545 CleanUnusedInitializersAndNodeArgs] Removing initializer '/embeddings/Gather_1_output_0'. It is no longer used by any node.
2023-03-30 16:28:45.468499139 [I:onnxruntime:Default, graph.cc:3545 CleanUnusedInitializersAndNodeArgs] Removing initializer '/embeddings/Mul_output_0'. It is no longer used by any node.
2023-03-30 16:28:45.468503339 [I:onnxruntime:Default, graph.cc:3545 CleanUnusedInitializersAndNodeArgs] Removing initializer '/embeddings/Transpose_1_output_0'. It is no longer used by any node.
2023-03-30 16:28:45.468507439 [I:onnxruntime:Defaul

RuntimeError: /onnxruntime_src/orttraining/orttraining/python/orttraining_pybind_state.cc:841 onnxruntime::python::addObjectMethodsForTraining(pybind11::module&, onnxruntime::python::ExecutionProviderRegistrationFn)::<lambda(onnxruntime::python::PyGradientGraphBuilder*)> [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Cannot compute the partial derivative for 'pooler.dense.bias' as it's unreachable from the output node(s).


In [28]:

onnxblock.save_checkpoint(training_block.parameters(), f"training_artifacts/{model_name}.ckpt")
onnx.save(onnx_model, f"training_artifacts/{model_name}_training.onnx")
onnx.save(eval_model, f"training_artifacts/{model_name}_eval.onnx")
onnx.save(optimizer_model, f"training_artifacts/{model_name}_optimizer.onnx")

## generating tokens

In [2]:
def tokenize_function(examples, pad_to_len):
    tokenizer = transformers.AutoTokenizer.from_pretrained("google/mobilebert-uncased")
    # filter out empty strings
    examples["text"] = [sent for sent in examples["text"] if len(sent) > 0]
    return tokenizer(examples["text"], return_special_tokens_mask=True, padding="max_length", max_length=pad_to_len, truncation=True)

def generate_tokens(corpus):
    """
    Takes in a Dataset with a "text" feature.

    Returns a Dataset with the following features: text, input_ids, token_type_ids, attention_mask, special_tokens_mask
    """
    # pad_to_len must be calculated before the batching happens to create consistent sizes in the resulting tensor
    # pad_to_len = max([len(sent) for sent in corpus["text"]])
    pad_to_len = 150 # shortened for demonstration purposes
    return corpus.map(tokenize_function, batched=True, fn_kwargs={"pad_to_len": pad_to_len})

def generate_json_dict(token_dataset):
    """
    Takes in a Dataset with the following features: text, input_ids, token_type_ids, attention_mask, special_tokens_mask

    Basically changes the 2d Python lists into two fields: a shape & a flattened list, for easier conversion to OnnxValues

    Returns a dictionary with the following keys: input_ids, input_size, token_type_ids, token_type_size, attention_mask, attention_mask_size, special_tokens_mask, special_tokens_size
    """
    json_dict = {}
    keys_to_convert = ["input_ids", "token_type_ids", "attention_mask", "special_tokens_mask"]

    for key_name in keys_to_convert:
        # add field for the shape of the tensor
        json_dict[key_name + "_shape"] = [len(token_dataset[key_name]), len(token_dataset[key_name][0])]
        # flatten list
        json_dict[key_name] = [num for sent in token_dataset[key_name] for num in sent]
    
    return json_dict


In [3]:
dataset_name = "wikitext" 
dataset_config = "wikitext-2-v1"
# corpus = type DatasetDict with three Datasets: test, train, validation
corpus = load_dataset(dataset_name, dataset_config)

Found cached dataset wikitext (/home/carolinezhu/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 733.57it/s]


In [15]:
#corpus["train"]["text"]
# max([len(sent) for sent in corpus["train"]["text"]])
# 3837
#len(corpus["train"]["text"])
# 36718

train_test = generate_tokens(corpus["train"])

In [16]:
print(len(train_test["input_ids"]))
# 36718
print(len(train_test["input_ids"][0]))
# 3837

max([len(sent) for sent in train_test["input_ids"]])

23767
150


150

In [4]:
test_tokens = generate_tokens(corpus["test"])
test_tokens = generate_json_dict(test_tokens)
train_tokens = generate_tokens(corpus["train"])
train_tokens = generate_json_dict(train_tokens)
validation_tokens = generate_tokens(corpus["validation"])
validation_tokens = generate_json_dict(validation_tokens)

Loading cached processed dataset at /home/carolinezhu/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b83d710dd1d17dd2.arrow
Loading cached processed dataset at /home/carolinezhu/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f91b7e5c1178149a.arrow
Loading cached processed dataset at /home/carolinezhu/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-aa24fc21c9608d06.arrow


In [ ]:
# write all the tokens to a json file
file_names = ["test_tokens.json", "train_tokens.json", "validation_tokens.json"]
token_dicts = [test_tokens, train_tokens, validation_tokens]

def write_dicts_to_files(file_names, dicts):
    # assumes file_names and dicts are 2 lists w/ the same lengths
    for i in range(len(file_names)):
        with open(file_names[i], "w") as json_file:
            json.dump(dicts[i], json_file)

write_dicts_to_files(file_names, token_dicts)